# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

Importando as bibliotecas a serem utilizadas:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from emoji import UNICODE_EMOJI

Lendo a planilha de treinamento:

In [3]:
treinamento = pd.read_excel('tweets_xbox_treinamento.xlsx')

Limpando a base de treinamento:

In [4]:
bad_char = [':', '.', ';', '"', '?', '!', '{', '}', '*', '$', '[', ']', '(', ')', '/' '&', '<', '>', "'"]
space_char = ['\n', '\t', ',']
tweets = []
treinamento_parcial = pd.DataFrame()

for tweet in treinamento.Tweets:
    x = ''
    for i in range(len(tweet)):
        if tweet[i] in UNICODE_EMOJI:
            x = x + ' ' + tweet[i] + ' '
        elif tweet[i] in space_char:
            x = x + ' '
        elif tweet[i] not in bad_char:
            x = x + tweet[i]
    tweets.append(x)
treinamento_parcial['Tweets'] = tweets
treinamento_parcial['Classificação'] = treinamento['Classificação']

In [5]:
treinamento_limpo = pd.DataFrame()
tweets = []
tweet_final = ' '
for tweet in treinamento_parcial.Tweets:
    limpa_palavras = []
    lista_palavras = tweet.split(' ')
    for palavra in lista_palavras:
        if palavra in UNICODE_EMOJI:
            limpa_palavras.append(palavra)
        elif len(palavra) > 2 and palavra[:4] != 'http' and palavra[0] != '@' \
        and palavra[0] != '#':
            limpa_palavras.append(palavra)
    tweets.append(tweet_final.join(limpa_palavras))
treinamento_limpo['Tweets'] = tweets
treinamento_limpo['Classificação'] = treinamento['Classificação']

Listando todas as palavras e contando o número de palavras que pertencem a tweets relevantes e não relevantes:

In [6]:
every_word = []

for i in range(len(treinamento_limpo['Tweets'])):
    lista_palavras = treinamento_limpo['Tweets'][i].split(' ')
    for palavra in lista_palavras:
        if palavra not in every_word:
            every_word.append(palavra)
            
words_relevante = 0
words_nao_relevante = 0

for i in range(len(treinamento_limpo['Tweets'])):
    lista_palavras = treinamento_limpo['Tweets'][i].split(' ')
    if treinamento_limpo['Classificação'][i] == 1:
        for palavra in lista_palavras:
            words_relevante += 1
    else:
        for palavra in lista_palavras:
            words_nao_relevante += 1

Dicionarios com a frequencia das palavras: (Laplace Smoothing já implementado)

In [7]:
relevante = {}
nao_relevante = {}

for word in every_word:
    relevante[word] = 1
    nao_relevante[word] = 1

for i in range(len(treinamento_limpo['Tweets'])):
    lista_palavras = treinamento_limpo['Tweets'][i].split(' ')
    if treinamento_limpo['Classificação'][i] == 1:
        for palavra in lista_palavras:
            relevante[palavra] += 1
    else:
        for palavra in lista_palavras:
            nao_relevante[palavra] += 1

Calculando a P( palavra | relevante ) e P( palavra | não relevante ):

In [8]:
P_relevante = {}
P_nao_relevante = {}

for palavra in every_word:
    P_relevante[palavra] = relevante[palavra] / (words_relevante + len(every_word))
    P_nao_relevante[palavra] = nao_relevante[palavra] / (words_nao_relevante + len(every_word))

Calculando P( relevante ) e P( não relevante ):

In [9]:
S_rel = 0
S_nrel = 0
for valor in treinamento_limpo['Classificação']:
    if valor == 1:
        S_rel += 1
    else:
        S_nrel += 1
        
P_rel = S_rel / len(treinamento_limpo['Classificação'])
P_nrel = S_nrel / len(treinamento_limpo['Classificação'])

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

Lendo a planilha de teste:

In [10]:
teste = pd.read_excel('tweets_xbox_teste.xlsx')

Limpando a planilha de teste:

In [11]:
bad_char = [':', '.', ';', '"', '?', '!', '{', '}', '*', '$', '[', ']', '(', ')', '/', '&', '<', '>', "'"]
space_char = ['\n', '\t', ',']
tweets = []
teste_parcial = pd.DataFrame()

for tweet in teste.Tweets:
    x = ''
    for i in range(len(tweet)):
        if tweet[i] in UNICODE_EMOJI:
            x = x + ' ' + tweet[i] + ' '
        elif tweet[i] in space_char:
            x = x + ' '
        elif tweet[i] not in bad_char:
            x = x + tweet[i]
    tweets.append(x)
teste_parcial['Tweets'] = tweets
teste_parcial['Classificação'] = teste['Classificação']

In [12]:
teste_limpo = pd.DataFrame()
tweets = []
tweet_final = ' '
for tweet in teste_parcial.Tweets:
    limpa_palavras = []
    lista_palavras = tweet.split(' ')
    for palavra in lista_palavras:
        if palavra in UNICODE_EMOJI:
            limpa_palavras.append(palavra)
        elif len(palavra) > 2 and palavra[:4] != 'http' and palavra[0] != '@' \
        and palavra[0] != '#':
            limpa_palavras.append(palavra)
    tweets.append(tweet_final.join(limpa_palavras))
teste_limpo['Tweets'] = tweets
teste_limpo['Classificação'] = teste['Classificação']

Prevendo se um tweet é relevante ou não:

In [13]:
previsao = []
for tweet in teste_limpo['Tweets']:
    prob_r = 1
    prob_nr = 1
    frase = tweet.split(' ')
    for palavra in frase:
        if palavra in P_relevante:
            prob_r *= P_relevante[palavra]
        else:
            prob_r *= (1 / (words_relevante + len(every_word)))
    for palavra in frase:
        if palavra in P_nao_relevante:
            prob_nr *= P_nao_relevante[palavra]
        else:
            prob_nr *= (1 / (words_nao_relevante + len(every_word)))
    relevant = prob_r * P_rel
    not_relevant = prob_nr * P_nrel
    if relevant > not_relevant:
        previsao.append(1)
    else:
        previsao.append(0)
    
teste_limpo['Previsão'] = previsao

Calculando as medidas pedidas:

In [14]:
prcr = 0
pnrcr = 0
prcnr = 0
pnrcnr = 0

for i in range(len(teste_limpo['Previsão'])):
    if teste_limpo['Previsão'][i] == 1 and teste_limpo['Classificação'][i] == 1:
        prcr += 1
    elif teste_limpo['Previsão'][i] == 0 and teste_limpo['Classificação'][i] == 1:
        pnrcr += 1
    elif teste_limpo['Previsão'][i] == 1 and teste_limpo['Classificação'][i] == 0:
        prcnr += 1
    else:
        pnrcnr += 1
        
P_pos_falso = prcnr / len(teste_limpo['Classificação'])
P_pos_verdadeiro = prcr / len(teste_limpo['Classificação'])
P_neg_verdadeiro = pnrcnr / len(teste_limpo['Classificação'])
P_neg_falso = pnrcr / len(teste_limpo['Classificação'])

print('Porcentagem de positivos falsos: {0:.2f}%'.format(P_pos_falso * 100))
print('Porcentagem de positivos verdadeiros: {0:.2f}%'.format(P_pos_verdadeiro * 100))
print('Porcentagem de negativos verdadeiros: {0:.2f}%'.format(P_neg_verdadeiro * 100))
print('Porcentagem de negativos falsos: {0:.2f}%'.format(P_neg_falso * 100))

Porcentagem de positivos falsos: 22.50%
Porcentagem de positivos verdadeiros: 13.50%
Porcentagem de negativos verdadeiros: 55.50%
Porcentagem de negativos falsos: 8.50%


In [15]:
acertos = 0
for i in range(len(teste_limpo['Previsão'])):
    if teste_limpo['Previsão'][i] == teste_limpo['Classificação'][i]:
        acertos += 1
        
accuracy = acertos / len(teste_limpo)

print('O classificador prevê a relevância de um tweet corretamente em {0:.2f}% dos casos.'.format(accuracy * 100))

O classificador prevê a relevância de um tweet corretamente em 69.00% dos casos.


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


**Comparação Qualitativa das Medidas Obtidas**

O classificador apresentou uma acurácia de 69%, o que significa que acertou a classificação de 69% dos tweets da base de teste. Além disso, apresentou precisão de 22.5% na classificação de tweets positivos falsos, ou seja, tweets relevantes marcados como irrelevantes e 13.5% de tweets positivos verdadeiros, ou seja, tweets relevantes marcados como tal. Em relação aos negativos falsos, ou seja, tweets irrelevantes marcados como relevantes, a precisão foi de 8.5% e em relação aos negativos verdadeiros, ou seja, tweets irrelevantes marcados como irrelevantes, a precisão foi de 55.5%. 

**Mensagens com Dupla Negação e Sarcasmo**

O classificador não é capaz de entender quando o tweet apresenta mensagens de dupla negação ou sarcasmo pois como o próprio nome do algoritmo (Naive-Bayes) já indica ('Naive' em inglês significa 'ingênuo'), este desconsidera a correlação entre as variáveis, ou seja, as analisa individualmente e não considera o que a presença delas, por exemplo, num mesmo tweet, significa. Desta forma, o classificador não é capaz de entender expressões que apresentem dupla negação, já que classificaria ambas as negações individualmente e não o que a presença das duas juntas significa e nem de identificar expressões sarcasticas, já que estas dependem de contexto e o algoritmo trabalha com variávies independentes.

**Plano de Expansão**

Este classificador é muito útil para as empresas pois facilita sua comunicação com o consumidor, já que reduz a quantidade total de tweets para menos que a metade, deixando com alta precisão apenas os tweets relevantes em relação a determinado produto, pode economizar tempo de um funcionário responsável por responder consumidores nas redes sociais e permite que a empresa encontre um feedback dos usuários de forma mais ágil. O financiamento do projeto permitiria que fossem feitas melhorias em relação à limpeza da base de dados e permitiria que fossem coletados um número maior de tweets, o que resultaria numa precisão ainda maior na seleção de tweets relevantes, tornando os processos de feedback ao usuário e absorção do feedback dado por este mais eficazes.


**Alimentação da Base de Treinamento**

Não é possível alimentar a base de dados do Treinamento automaticamente utilizando o próprio classificador, aplicado a novos tweets, pois este não possui conhecimento suficiente para classificar corretamente qualquer tipo de tweet. Desta forma, a classificação dos novos tweets seria enviesada, devido à incapacidade do classificador de identificar expressões que apresentem, por exemplo, dupla negação e sarcasmo. 


**Diferentes Cenários para o Uso de Naive-Bayes**

O Naive-Bayes é um algoritmo que pode ser utilizado com diversas finalidades, mas sempre trabalhando com probabilidade de variáveis. Desta forma, pode ser utilizado, por exemplo, para identificar a partir de uma descrição qual ser vivo está sendo descrito. Neste caso, no entanto, não poderiam haver seres com mesma característica, já que o algoritmo é incapaz de classificar baseado em um conjunto de variáveis, analisando independentemente cada uma delas. Ou seja, se o usuário deseja, por exemplo, saber se certa descrição se refere a um tipo de animal ou a um tipo de planta, ele pode usar o Naive-Bayes para calcular a probabilidade de cada palavra da descrição ser referente a cada um dos seres e chegar a uma resposta. 
 

